In [ ]:
import cvlib as cv
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageFont, ImageDraw, Image
from urllib.request import urlopen
import requests
from http.client import IncompleteRead

url="http://192.168.25.51:81/stream"
cap = cv2.VideoCapture(url)
model = load_model('saved_model_mayhun.h5')
oldURL = " "

while(True):
    # Capture frame-by-frame
    status, frame = cap.read()
    
    if not status:
        print("Could not read frame")
        exit()
 
    # apply face detection
    face, confidence = cv.detect_face(frame)
 
    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:
            
            face_region = frame[startY:endY, startX:endX]
            
            face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)
            
            x = img_to_array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            prediction = model.predict(x)
 
            if prediction < 0.5: # 마스크 미착용으로 판별되면, 
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "No Mask ({:.2f}%)".format((1 - prediction[0][0])*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                URL = "http://192.168.25.51/nomask"  
                
            else: # 마스크 착용으로 판별되면
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "Mask ({:.2f}%)".format(prediction[0][0]*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                URL = "http://192.168.25.51/mask"
                
            if oldURL != URL:
                try:
                    response = requests.get(URL)
                    print("request sent")
                except IncompleteRead:
                    print("request could not be sent")
            

    # display output
    cv2.imshow("mask nomask classify", frame)
 
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

1/1 [==============================] - 1s 903ms/step
request sent
1/1 [==============================] - 0s 117ms/step
request sent
1/1 [==============================] - 0s 115ms/step
request sent
1/1 [==============================] - 0s 113ms/step
request sent
1/1 [==============================] - 0s 112ms/step
request sent
1/1 [==============================] - 0s 118ms/step
request sent
1/1 [==============================] - 0s 124ms/step
request sent
1/1 [==============================] - 0s 117ms/step
request sent
1/1 [==============================] - 0s 113ms/step
request sent
1/1 [==============================] - 0s 116ms/step
request sent
1/1 [==============================] - 0s 112ms/step
request sent
1/1 [==============================] - 0s 111ms/step
request sent
1/1 [==============================] - 0s 114ms/step
request sent
1/1 [==============================] - 0s 115ms/step
request sent
1/1 [==============================] - 0s 111ms/step
request sent
1/1 [=====